# SEI Composition generator/fitting code

In this code, we're going to randomly generate a series of SEI compositions, to determine what range of compositions are consistent with a fit to some neutron reflectometry (NR) measurements.

 We're going to take the best fit value and 68% confidence interval from some neutron reflectometry (NR) fits for the SEI layer thicknesses and scattering length densities (SLDs), along with the total SEI mass (measured by quartz crystal microbalance, QCM).

 We then generate a random SEI composition, and see if it is consistent with the experimental values.  If it is, we add it to a collection of 'feasible' models (if it is not, we discard it). By plotting the 'feasible' models, we can get an idea of the SEI composition.

 Note that this approach is necessitated because the composition is under-determined from the data at hand: there are more possible components than there are constraining variables (otherwise, we would just calculate composition directly from the fit + QCM).


# First, import dependencies:

In [1]:
import numpy as np
from numpy import random as rand
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats.mstats import mquantiles
from scipy.stats import norm
import time
from multiprocessing import Pool

# User inputs:

In [2]:
n_comp_target = 100000      #Target number of accepted compositions
n_process = 8               #Number of processors to run simultaneously for parallelization.

# SLDs from the NR fit.
#   inner layer:
min_SLD_in = 1.324;		# 68% CI lower bound
max_SLD_in = 1.533;		# 68% CI upper bound
mean_SLD_in = 1.404;   	# Mean SEI SLD (i.e. best fit)
#   outer layer:
min_SLD_out = 3.669;    # 68% CI lower bound
max_SLD_out = 3.779;	# 68% CI upper bound
mean_SLD_out = 3.748;   # Mean SEI SLD (i.e. best fit)

# Thicknesses (angstroms)
#   inner layer:
th_min_in = 35.0;		# 68% CI lower bound
th_max_in = 40.2;		# 68% CI upper bound
th_mean_in = 36.71;   	# Mean inner thickness (i.e. best fit)
#   outer layer:
th_min_out = 144.0;		# 68% CI lower bound
th_max_out = 179.6;		# 68% CI upper bound
th_mean_out = 157.36;   # Mean outer thickness (i.e. best fit)

# Mass (ng/cm2)
#   There is, thus far, a lot of inter-experiment variability in the mass uptake
#   The range below represents the mean +/- one standard deviation from a set of 4
#   separate experiments.
m_min = 771.0;
m_max = 3811.0;

elyte_relDensity = 0.87658;   # Fitted electrolyte relative density (% of bulk)

# Upper y limit for plotting
nmax = 0.6
# Upper x limit for plotting: plot the whole range
xmax = 1.0

Perform some calculatios to estimate standard deviations:

In [3]:
std_SLD_in = ((mean_SLD_in-min_SLD_in)+(max_SLD_in-mean_SLD_in))/2;
std_SLD_in_inv = 1/std_SLD_in;
std_SLD_out = ((mean_SLD_out-min_SLD_out)+(max_SLD_out-mean_SLD_out))/2;
std_SLD_out_inv = 1/std_SLD_out;
std_th_in = ((th_mean_in-th_min_in)+(th_max_in-th_mean_in))/4;
std_th_out = ((th_mean_out-th_min_out)+(th_max_out-th_mean_out))/4;

# SEI Component Properties

Specify the neutron SLD and mass density for each possible component.

In [3]:
# SLDs: 10^-4 nm^-2
Li_SLD = -0.880;
LiOH_SLD = 0.060;
Li2O_SLD = 0.812;
LiAlkyl_SLD = 2.22;
LiF_SLD = 2.301;
LEDC_SLD = 2.95;
W_SLD = 3.065;
Li2CO3_SLD = 3.485;
elyte_SLD = 4.558*elyte_relDensity;
LEC_D_SLD = 8.656;

# Densities: g/cm3
Li_rho = 0.534;
Li2O_rho = 2.01;
LiOH_rho = 1.46;
Li2CO3_rho = 2.11;
LiF_rho = 2.64;
LEC_D_rho = 2.22;
LEDC_rho = 1.86;
LEDC_D_rho = 1.91;
LiAlkyl_rho = 2.11;
W_rho = 19.25;
elyte_rho = 1.329*elyte_relDensity;

# Load all values into arrays, which we'll use to calculate the SLD and mass 
#     of the proposed inner and outer SEI layers:
SLDs = np.array([Li_SLD, LiOH_SLD, Li2O_SLD, LiAlkyl_SLD, LiF_SLD, LEDC_SLD,
                 Li2CO3_SLD, elyte_SLD, LEC_D_SLD]);

densities_in = np.array([Li_rho, LiOH_rho, Li2O_rho, LiAlkyl_rho, LiF_rho, 
                         LEDC_rho,  Li2CO3_rho, elyte_rho, LEC_D_rho]);

# Assume elyte does not contribute to QCM mass uptake for outer SEI layer:
densities_out = np.array([Li_rho, LiOH_rho, Li2O_rho, LiAlkyl_rho, LiF_rho, 
                         LEDC_rho, Li2CO3_rho, 0.0, LEC_D_rho]);

# Number of chemical components in the model:
n_elements = len(SLDs)

# Composition generation and evaluation function:

Here's where we will loop over randomly generated compositions until we have found n_comp_target which are consistent with the data.  For parallel calculations, we will divide this into `n_process` `while` loops, each of which will collect $\frac{{\tt n\_comp\_target}}{{\tt n\_process}}$ compositions.

In [8]:
def Looper(n_comp_target):

    # Initialize the list of 'feasible' compositions:
    composition_list = np.zeros((int(n_comp_target),2*n_elements+5))

    # Initialize n_comp: the number of accepted compositions:
    n_comp = 0

    while n_comp < n_comp_target:
        # Randomly generate total thicknesses from normal distribution:
        th_total = np.array((th_mean_in+rand.randn()*std_th_in,
            th_mean_out+rand.randn()*std_th_out));

        #   Generate random composition for inner SEI
        randarray = rand.rand(n_elements)
        #randarray[4:6] = np.zeros((1,2)) # no d-LEC or LEDC in inner layer
        comp_in = randarray/np.sum(randarray)   #Volume fractions of each component
        thicknesses_in = th_total[0]*comp_in    #Partial thickness of each component

        # Repeat for outer SEI:
        #randarray = np.zeros(n_elements)
        #randarray[4:7] = rand.rand(3)
        randarray = rand.rand(n_elements)
        comp_out = randarray/np.sum(randarray)
        thicknesses_out = th_total[1]*comp_out

        # Calculate SLD of the resulting layers:
        comp_SLD = np.array((np.dot(comp_in,SLDs),np.dot(comp_out,SLDs)))

        # Calculate summed mass of the 2 layers:
        mass_total = np.dot(10*thicknesses_in,densities_in) + \
            np.dot(10*thicknesses_out,densities_out);

        # Calculate probability of observing:
        #   1. Calculated SLD of inner layer
        #   2. Calculated SLD of outer layer
        #
        # Modified Matropolis-Hastings algorithm: If both probabiliites are higher than
        #   a randomly chosen number and the mass falls wihtin an, store this composition 
        #   and increment n_comp.
        if (rand.random() < 2*(1-norm.cdf(abs(comp_SLD[1]-mean_SLD_out)*std_SLD_out_inv))
            and rand.random() < 2*(1-norm.cdf(abs(comp_SLD[0]-mean_SLD_in)*std_SLD_in_inv))
            and mass_total < m_max and mass_total > m_min):

            composition_list[n_comp,:] = np.hstack((comp_in, comp_out, comp_SLD, th_total, mass_total))
            n_comp += 1
            # Status update: print out every time n_comp reaches a mult of 500
            if not (n_comp % 50):
                print('n_comp = ',n_comp)

    # The function returns the list of compositions consistent with the data:
    return composition_list

# Set up for parallel processing and run:

In [9]:
# Start a timer, just to see how long it takes to run the composition generator:
start = time.time()

# Create a pool of processes:
pool = Pool(processes=n_process)

# Run 'Looper' in parallel over the pool
composition_list = pool.map(Looper,n_comp_target/n_process*np.ones(n_process))

# Stack the results from teh pool into a single array:
composition_list = np.vstack(composition_list)

# How long did it take to run the composition generator?
sim_end = time.time()
print("Simulation time = %1.2f minutes"%((sim_end-start)/60.0))

n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  50
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  100
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  150
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  200
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  250
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  300
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  350
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  400
n_comp =  450
n_comp =  450
n_comp =  450
n_comp =  450
n_comp =  450
n_comp =  450
n_comp =  450
n_comp =  450


n_comp =  3500
n_comp =  3500
n_comp =  3500
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3550
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3600
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3650
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3700
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3750
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3800
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3850
n_comp =  3900
n_comp =  3900
n_comp =  3900
n_comp =  3900
n_comp =  3900
n_comp =  3900
n_comp =  3900
n_comp =  

n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  6950
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7000
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7050
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7100
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7150
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7200
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7250
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7300
n_comp =  7350
n_comp =  7350
n_comp =  

n_comp =  10300
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10350
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10400
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10450
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10500
n_comp =  10550
n_comp =  10550
n_comp =  10550
n_comp =  10550
n_comp =  10550
n_comp =  10550
n_comp =  10550
n_comp =  10600
n_comp =  10550
n_comp =  10600
n_comp =  10600
n_comp =  10600
n_comp =  10600
n_comp =  10600
n_comp =  10600
n_comp =  10600
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10650
n_comp =  10700
n_comp =  10700
n_comp =  10700
n_comp =  10700
n_comp =  10700
n_comp =

# Configure plotting options:

In [4]:
ff = 10     # font size
lw = 2.0    # line width

nbins = 50  # Number of bins for histogram

font = plt.matplotlib.font_manager.FontProperties(family='Times New Roman',
    size=ff)
cmap = plt.get_cmap('plasma')
ndata = 10
color_ind = np.linspace(0,1,ndata)
colors = list()
for i in np.arange(ndata):
    colors.append(cmap(color_ind[i]))

# Inner SEI plot color
ci = colors[2]
# Outer SEI plot color
co = colors[7]

# Set up the histogram bins:
binwidth = 1./nbins
binstat = 1./nbins
small = 1e-6  # Any volume fraciton below this is considered = 0.
cut_points = np.hstack([0.0, small, np.arange(binstat,1.+binstat,binstat)])

binNames = np.hstack([0.0, np.arange(binstat,1.+binstat,binstat) - binstat/2.0])


# The chemical names of the SEI components to display:
ChemNames = ('Inner Lithium', 
             'Inner LiOH',
             'Inner Li2O',
             'Inner LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$',
             'Inner LiF', 
             'Inner LEDC', 
             'Inner Li'+'$_2$'+'CO'+'$_3$',
             'InnerPorosity',
             'Inner d-LEC',
             'Outer Lithium', 
             'Outer LiOH',
             'Outer Li2O',
             'Outer LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$',
             'Outer LiF', 
             'Outer LEDC', 
             'Outer Li'+'$_2$'+'CO'+'$_3$',
             'OuterPorosity',
             'Outer d-LEC',
             'SLD_inner', 'SLD_outer', 't_inner', 't_outer', 'Mass')


# Define the plotting function:

In [5]:
def histplot(ax,data,species,index,color,layer,savename,plotname,xtext=0.45):

    # Sort the range of feasible volume fractions into bins:

    binnedData = pd.cut(data[species],bins=cut_points,labels=binNames, include_lowest=True)            
        
    #print(binnedData)    
    # Count how many are in each bin:
    Sorted = pd.value_counts(binnedData,ascending=True)

    dataRange = data[species].max() - data[species].min()
    nbins = max(1,int(dataRange/binwidth))

    val, weight = np.array(list(zip(*[(k, v) for k,v in Sorted.items()])))

    # Total number of compositions:
    npop = sum(weight)/100

    # Plot histogram:
    ax.hist(val, weights=weight/npop, bins = int(1.0/binwidth),color=color, edgecolor='k')
    
    # Set limits, add annotation:
    plt.xlim(0,xmax)
    plt.ylim(0,nmax*100)
    
    ax.text(xtext,82.5*nmax,plotname,fontsize=ff-2, family='Times New Roman')
    ax.text(xtext,66.*nmax,"SLD = %1.2f"%SLDs[index]+ r"$\times$"+"10"+r"$^{-4}$" +" nm"+r"$^{\rm -2}$" ,fontsize=ff-2, family='Times New Roman')
    ax.text(xtext,50.*nmax,"Density = %1.2f"%densities_in[index]+" g cm"+r"$^{-3}$",fontsize=ff-2, family='Times New Roman')

    ax.get_yaxis().set_tick_params('both', direction='in',pad=5.0)
    ax.get_xaxis().set_tick_params('both', direction='in',pad=5.0)
    
    ax.set_yticks([0,25,50])
    
    # Format tick labels:
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(ff-2)
        tick.label1.set_fontname('Times New Roman')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(ff-2)
        tick.label1.set_fontname('Times New Roman')



# Plot the results:

In [9]:
# Convert the array of 'feasible' compositions to a Pandas dataFrame:
#data = pd.DataFrame(data=composition_list,columns=ChemNames)
data = pd.read_csv('SEI_Comp_Model_06102018.csv')


# Create plot of inner SEI compositions:
f, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(9, sharex=True, sharey=True)

f.set_size_inches((3.0,6.75))
plt.tight_layout(h_pad=0.0)
f.subplots_adjust(hspace=0)
plt.xlabel('Volume Fraction',fontsize=ff,family='Times New Roman')
ax5.set_ylabel('% of Models',fontsize=ff,labelpad=1.0,family='Times New Roman')
#print(data)

# Call the plotting function for this model, one component at a time:
histplot(ax1,data,'Inner Lithium',0,ci,'Inner','Lithium','Li')
histplot(ax2,data,'Inner LiOH',1,ci,'Inner','LiOH','LiOH')
histplot(ax3,data,'Inner Li2O',2,ci,'Inner','Li2O','Li'+'$_2$'+'O')
histplot(ax4,data,'Inner LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$',3,ci,
    'Inner','LiAlkylCarbonate','LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$')
histplot(ax5,data,'Inner LiF',4,ci,'Inner','LiF','LiF')
histplot(ax6,data,'Inner LEDC',5,ci,'Inner','LEDC','LEDC')
histplot(ax7,data,'Inner Li'+'$_2$'+'CO'+'$_3$',6,ci,'Inner','Li2CO3',
    'Li'+'$_2$'+'CO'+'$_3$')
histplot(ax8,data,'InnerPorosity',7,ci,'Inner','Porosity',
    'Porosity')
histplot(ax9,data,'Inner d-LEC',8,ci,'Inner','d-LEC','d-LEC')

# Save figure:
plt.savefig('../InnerSEI_hist.pdf',dpi=350)

# Close figure:
plt.close()

# Create plot of outer SEI compositions:
f, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(9, sharex=True, sharey=True)

f.set_size_inches((3.0,6.75))
plt.tight_layout(h_pad=0.0)
f.subplots_adjust(hspace=0)
plt.xlabel('Volume Fraction',fontsize=ff,family='Times New Roman')
ax5.set_ylabel('% of Models',fontsize=ff,labelpad=1.0,family='Times New Roman')

# Call the plotting function for this model, one component at a time:
# Outer Layer
histplot(ax1,data,'Outer Lithium',0,co,'Outer','Lithium','Li')
histplot(ax2,data,'Outer LiOH',1,co,'Outer','LiOH','LiOH')
histplot(ax3,data,'Outer Li2O',2,co,'Outer','Li2O','Li'+'$_2$'+'O')
histplot(ax4,data,'Outer LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$',3,co,
    'Outer','LiAlkylCarbonate','LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$')
histplot(ax5,data,'Outer LiF',4,co,'Outer','LiF','LiF')
histplot(ax6,data,'Outer LEDC',5,co,'Outer','LEDC','LEDC')
histplot(ax7,data,'Outer Li'+'$_2$'+'CO'+'$_3$',6,co,'Outer','Li2CO3',
    'Li'+'$_2$'+'CO'+'$_3$')
histplot(ax8,data,'OuterPorosity',7,co,'Outer','Porosity',
    'Porosity')
histplot(ax9,data,'Outer d-LEC',8,co,'Outer','d-LEC','d-LEC')


# Save figure:
plt.savefig('../OuterSEI_hist.pdf',dpi=350)

# Close figure:
plt.close()


## Calculate the mpe of the fitted model, using the mode of each component distribution

The array of constituent molecule/phase charge numbers and molecular weights are in the following order:

    Li2O, LiOH, Li2CO3, LiF, d-LEC, LEDC, elyte, Li, LiAlkyl

In [7]:
data = pd.read_csv('SEI_Comp_Model_06102018.csv')

mass = np.mean(data['Mass'])

# Mols of electrons transfered per mole of SEI component:
n_elec = np.array([1, 1, 1, 1, 1, 2, 1, 0, 1])

# Molecular weight of each species (g/mol):
#MW = np.array([29.88, 23.95, 73.89, 25.94, 96.01, 101.18, 161.95, 6.94, 96.01])
MW = np.array([6.94, 23.95, 29.88, 96.01, 25.94, 101.18, 73.89, 161.95, 96.01])

t_k_inner = list()
t_k_outer = list()

def PartialThickness(name,layer):
    t_k = data[name]*data[layer]
    
    """This is some old code which calculated the mode. We found that the mean
    was a better representation, but we preserve the code here, for posterity:
    
    t_max = np.max(data[layer])
    
    cut_points = np.hstack([0.0, t_max*small, t_max*np.arange(binstat,1.+binstat,binstat)])

    binNames = np.hstack([0.0, t_max*np.arange(binstat,1.+binstat,binstat) - binstat/2.0])
    
    binnedData = pd.cut(t_k,bins=cut_points,labels=binNames, include_lowest=True)
    
    # Count how many are in each bin:
    Sorted = pd.value_counts(binnedData,ascending=True)
    
    # Calculate the mode:
    mode = np.argmax(Sorted)
    """
    
    mean = np.mean(t_k)
    return mean


# Calculate the mean of the partial thickness (t*V_k) distribution for each component:
layer = 't_inner'

name = 'Inner Li2O'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner LiOH'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner Li'+'$_2$'+'CO'+'$_3$'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner LiF'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner d-LEC'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner LEDC'
t_k_inner.append(PartialThickness(name,layer))

name = 'InnerPorosity'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner Lithium'
t_k_inner.append(PartialThickness(name,layer))

name = 'Inner LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$'
t_k_inner.append(PartialThickness(name,layer))



layer = 't_outer'

name = 'Outer Li2O'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer LiOH'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer Li'+'$_2$'+'CO'+'$_3$'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer LiF'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer d-LEC'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer LEDC'
t_k_outer.append(PartialThickness(name,layer))

name = 'OuterPorosity'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer Lithium'
t_k_outer.append(PartialThickness(name,layer))

name = 'Outer LiOCO'+'$_2$'+'C'+'$_2$'+'H'+'$_5$'
t_k_outer.append(PartialThickness(name,layer))

# Partial thicknesses in the denominator are multiplied by n_elec
denom_fac_inner = n_elec*densities_in/MW
denom_fac_outer = n_elec*densities_out/MW

#mass = np.dot(t_k_inner,densities_in)+np.dot(t_k_outer,densities_out)
print(mass)

# Calculate and print the MPE:
mpe = (np.dot(t_k_inner,densities_in)+np.dot(t_k_outer,densities_out))/(np.dot(t_k_inner,denom_fac_inner) + np.dot(t_k_outer,denom_fac_outer))

# Print the MPE:
print('Simulated m.p.e. is: ', mpe, 'g/mol-e')

3299.6739076772446
Simulated m.p.e. is:  36.563671541145666 g/mol-e


# Save the data

In [111]:
data.to_csv('SEI_Comp_Model_06102018.csv')